In [ ]:
!wget 'https://downloader.disk.yandex.ru/disk/3658fae4995ded2871a33feefd6116fecac03d7021a937532da1e8fff3ee655d/68123aab/9gpobZtiS6b_LxPKT93qSWAvQU8wwI14tn4XIBFr8z7OunNDB93OaiVDP3giJuQf_C6AKWKAqtL1YGDItD6_Qg%3D%3D?uid=0&filename=dataset.zip&disposition=attachment&hash=9WZjhXFgMHITKmDMlCgCYFFl369D/AuXyrHXWduxt5TQnd4SQ%2BhFsSShb7LkohwDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=60537076&fsize=888676148&hid=9f3b0ddc8cfd5de26d7719c8574227b8&media_type=compressed&tknv=v2' -O dataset.zip

--2025-04-30 15:16:54--  https://downloader.disk.yandex.ru/disk/3658fae4995ded2871a33feefd6116fecac03d7021a937532da1e8fff3ee655d/68123aab/9gpobZtiS6b_LxPKT93qSWAvQU8wwI14tn4XIBFr8z7OunNDB93OaiVDP3giJuQf_C6AKWKAqtL1YGDItD6_Qg%3D%3D?uid=0&filename=dataset.zip&disposition=attachment&hash=9WZjhXFgMHITKmDMlCgCYFFl369D/AuXyrHXWduxt5TQnd4SQ%2BhFsSShb7LkohwDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=60537076&fsize=888676148&hid=9f3b0ddc8cfd5de26d7719c8574227b8&media_type=compressed&tknv=v2
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 410 Gone
2025-04-30 15:16:55 ERROR 410: Gone.



In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of dataset.zip or
        dataset.zip.zip, and cannot find dataset.zip.ZIP, period.


In [ ]:
import torch
from torch import nn
import numpy as np
import glob
from matplotlib.image import imread
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
import torchvision.transforms as transforms

In [ ]:
!pip install timm

In [ ]:
import timm
from PIL import Image

In [ ]:
model = timm.create_model('vit_base_patch16_224.dino', pretrained=True)

model.eval()

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embeddings=[]
device='cuda'
model=model.to(device)
image_paths=glob.glob('./dataset/*.png')
for path in image_paths:
  with torch.no_grad():
    image = Image.open(path).convert('RGB')
    image = transforms(image)
    embeddings.append(model((image.unsqueeze(0)).to(device)))

In [ ]:
for i in range(len(embeddings)):
  embeddings[i]=embeddings[i].cpu()
embeddings=np.array(embeddings)

In [ ]:
embeddings.shape

(9605, 1, 768)

In [ ]:
embeddings=embeddings.reshape(9605,768)

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=7)
knn.fit(embeddings)
distances, indices = knn.kneighbors(embeddings)
closest_indices = indices[:, 1:]
closest_pictures = []
ansl=[]
for i, image_path in enumerate(image_paths):
    closest_paths = [image_paths[j][10:] for j in closest_indices[i]]
    path=image_path[10:]+','+' '.join(closest_paths)+'\n'
    ansl.append(path)
ansl=sorted(ansl, key=lambda x: int(x[:4]))

In [ ]:
f=open('ans.csv', 'w')
f.write('filename,ranking\n')
for i in ansl:
  f.write(i)

f.close()

In [ ]:
closest_pictures

[['6567.png', '5514.png', '1297.png', '3592.png', '2624.png', '8377.png'],
 ['5142.png', '7561.png', '6091.png', '0810.png', '4238.png', '7401.png'],
 ['1197.png', '6982.png', '9441.png', '0322.png', '5043.png', '9266.png'],
 ['1870.png', '1842.png', '6356.png', '3228.png', '8809.png', '5308.png'],
 ['3118.png', '7203.png', '5138.png', '2627.png', '4197.png', '0663.png'],
 ['1805.png', '3791.png', '1243.png', '3069.png', '1569.png', '4451.png'],
 ['0776.png', '2412.png', '4905.png', '7620.png', '2692.png', '9227.png'],
 ['4571.png', '6761.png', '0673.png', '5934.png', '4304.png', '2990.png'],
 ['3938.png', '7952.png', '6754.png', '8164.png', '4669.png', '5159.png'],
 ['7060.png', '4658.png', '3358.png', '6071.png', '7005.png', '5688.png'],
 ['8955.png', '8678.png', '4462.png', '6303.png', '3298.png', '3102.png'],
 ['3176.png', '0640.png', '0420.png', '0475.png', '3701.png', '8458.png'],
 ['3663.png', '1560.png', '5236.png', '3192.png', '8564.png', '3538.png'],
 ['4846.png', '9298.png',